In [2]:
from app.scheduler import job
import logging
logger = logging.getLogger(__name__)
await job()

Processing BOLs from AMAZON FREIGHT & CENTRAL FREIGHT (11 ORDERS) 05.30.24 (LIFEPRO)-1-2.pdf: 100%|██████████| 2/2 [00:08<00:00,  4.38s/it]
100%|██████████| 1/1 [00:16<00:00, 16.30s/it]


In [2]:
from tqdm import tqdm
from utils.ocr import OCRTool
from utils.pdf_splitter import process_pdf
from app.config import settings
from utils.emailclient import EmailAttachmentExtractor
from datetime import datetime, timedelta
from utils.sheet import SheetsClient
from utils.drive import DriveClient
import pandas as pd
import logging
logger = logging.getLogger(__name__)

In [2]:
email_client = EmailAttachmentExtractor(
        email_address=settings.EMAIL_ADDRESS,
        password=settings.EMAIL_PASSWORD,
        imap_server=settings.IMAP_SERVER
        )
if email_client.connect():
    today = datetime.now().strftime("%d-%b-%Y")
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%d-%b-%Y")
    pdfs = email_client.extract_pdf_attachments(num_emails=200,
                                            subject_contains=settings.WORD_IN_SUBJECT)
                                            # date_from=yesterday,
                                            # date_to=today)

Connected to email server


In [11]:
results = []
try:
    for pdf in pdfs:
        result = process_pdf(pdf['binary_data'], OCRTool(), pdf['file_name'])
        results.append(result)
        try:
            drive_client = DriveClient(credentials_file_path=settings.CREDENTIALS_FILE_PATH)
            for bol in result:
                file_name = bol.get('file_name')
                binary_data = bol.get('binary_data')
                file_path = 'folder1/' + file_name
                pdf_link = drive_client.upload_pdf(binary_data, file_path, parent_folder_id = settings.DRIVE_FOLDER_ID)
                bol['pdf_link'] = pdf_link

            data = [
                {
                    'ship_from_company_name': item['shipment_info']['ship_from']['company_name'],
                    'ship_from_contact_person': item['shipment_info']['ship_from']['contact_person'],
                    'ship_from_contact_number': item['shipment_info']['ship_from']['contact_number'],
                    'ship_from_address': item['shipment_info']['ship_from']['address'],
                    'ship_to_company_name': item['shipment_info']['ship_to']['company_name'],
                    'ship_to_contact_person': item['shipment_info']['ship_to']['contact_person'],
                    'ship_to_contact_number': item['shipment_info']['ship_to']['contact_number'],
                    'ship_to_address': item['shipment_info']['ship_to']['address'],
                    'carrier_name': item['shipment_info']['carrier_info']['carrier_name'],
                    'scac': item['shipment_info']['carrier_info']['scac'],
                    'pro_number': item['shipment_info']['carrier_info']['pro_number'],
                    'order_number': item['shipment_info']['customer_order_information']['order_number'],
                    'shipment_id': item['shipment_info']['customer_order_information']['shipment_id'],
                    'pallets': item['shipment_info']['customer_order_information']['pallets'],
                    'cartons': item['shipment_info']['customer_order_information']['cartons'],
                    'weight': item['shipment_info']['customer_order_information']['weight'],
                    'pdf_link': item['pdf_link'],
                } 
                for item in result
            ]

            result_dataframe = pd.DataFrame(data)
            sheets_client = SheetsClient(credentials_file_path=settings.CREDENTIALS_FILE_PATH)
            sheets_client.add_dataframe(
                                data_frame=result_dataframe,
                                sheet_name=settings.SHEET_NAME,
                                spreadsheet_name=settings.SPREADSHEET_NAME
                            )
        except Exception as e:
            logger.error(f"Error processing pdf: {e}")
            continue

except Exception as e:
    logger.error(f"Error processing pdf: {e}")
    email_client.disconnect()

Processing BOLs from AMAZON FREIGHT & CENTRAL FREIGHT (11 ORDERS) 05.30.24 (LIFEPRO).pdf: 100%|██████████| 11/11 [00:51<00:00,  4.68s/it]
Processing BOLs from chai6.11.pdf: 100%|██████████| 15/15 [01:03<00:00,  4.26s/it]


In [4]:
result1 = results[0]

drive_client = DriveClient(credentials_file_path=settings.CREDENTIALS_FILE_PATH)
for bol in result1:
    file_name = bol.get('file_name')
    binary_data = bol.get('binary_data')
    file_path = 'folder1/' + file_name
    pdf_link = drive_client.upload_pdf(binary_data, file_path, parent_folder_id = settings.DRIVE_FOLDER_ID)
    bol['pdf_link'] = pdf_link

data = [
    {
        'ship_from_company_name': item['shipment_info']['ship_from']['company_name'],
        'ship_from_contact_person': item['shipment_info']['ship_from']['contact_person'],
        'ship_from_contact_number': item['shipment_info']['ship_from']['contact_number'],
        'ship_from_address': item['shipment_info']['ship_from']['address'],
        'ship_to_company_name': item['shipment_info']['ship_to']['company_name'],
        'ship_to_contact_person': item['shipment_info']['ship_to']['contact_person'],
        'ship_to_contact_number': item['shipment_info']['ship_to']['contact_number'],
        'ship_to_address': item['shipment_info']['ship_to']['address'],
        'carrier_name': item['shipment_info']['carrier_info']['carrier_name'],
        'scac': item['shipment_info']['carrier_info']['scac'],
        'pro_number': item['shipment_info']['carrier_info']['pro_number'],
        'order_number': item['shipment_info']['customer_order_information']['order_number'],
        'shipment_id': item['shipment_info']['customer_order_information']['shipment_id'],
        'pallets': item['shipment_info']['customer_order_information']['pallets'],
        'cartons': item['shipment_info']['customer_order_information']['cartons'],
        'weight': item['shipment_info']['customer_order_information']['weight'],
        'pdf_link': item['pdf_link'],
    } 
    for item in result1
]

result1_dataframe = pd.DataFrame(data)
sheets_client = SheetsClient(credentials_file_path=settings.CREDENTIALS_FILE_PATH)
sheets_client.add_dataframe(
                    data_frame=result1_dataframe,
                    sheet_name=settings.SHEET_NAME,
                    spreadsheet_name=settings.SPREADSHEET_NAME
                )


In [1]:
from utils.pdf_splitter import split_pdf

pdf_file = 'test_documents/AMAZON FREIGHT & CENTRAL FREIGHT (11 ORDERS) 05.30.24 (LIFEPRO).pdf'

with open(pdf_file, 'rb') as file:
    binary_data = file.read()

result = split_pdf(binary_data)


In [2]:
with open('test.pdf', 'wb') as file:
    file.write(result[0]['pdf'])